In [2]:
import numpy as np
import scipy as sp
from scipy.stats import multivariate_normal
from scipy       import optimize
import scipy
import skimage
import matplotlib.pyplot as plt
import imageio
import pylab
from glob import glob
import sys
import itk
from itkwidgets import view
import math

In [3]:
num_mask = 3
im_mask = []
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth5_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth12_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth16_Edited.png", itk.F)))
im_mask_depth = [5,12,16]

In [4]:
def get_top_and_bottom_curve_points_C52(im, centerX):
    """ Find points along the top (yt) and bottom (yb) curves of the ultrasound image """
    min_x = int(centerX-centerX*0.1)
    max_x = int(centerX+centerX*0.1)
    step_x = 10
    size_x = int((max_x - min_x)/step_x)
    xt = np.zeros(size_x, dtype=int)
    yt = np.zeros(size_x, dtype=int)
    for t in range(size_x):
        xt[t] = int(min_x + t * step_x)
        mid = np.mean(im[:,xt[t]-2:xt[t]+2],axis=1)
        nz = np.flatnonzero(mid)
        yt[t] = nz[0]
    min_x = int(centerX-centerX*0.25)
    max_x = int(centerX+centerX*0.25)
    step_x = 10
    size_x = int((max_x - min_x)/step_x)
    xb = np.zeros(size_x, dtype=int)
    yb = np.zeros(size_x, dtype=int)
    for t in range(size_x):
        xb[t] = int(min_x + t * step_x)
        mid = np.mean(im[:,xb[t]-2:xb[t]+2],axis=1)
        nz = np.flatnonzero(mid)
        yb[t] = nz[nz.size-1]
    return xt,yt,xb,yb

def calculate_radius(xc, yc, x, y):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def calculate_radius_C52(yc, x, y, centerX):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-centerX)**2 + (y-yc)**2)

def fit_circle(c, x, y):
    """ calculate the algebraic distance between the 2D points and the mean circle centered at c=(xc, yc) """
    Ri = calculate_radius(*c, x, y)
    return Ri - Ri.mean()

def fit_circle_C52(c, x, y, centerX):
    """ calculate the algebraic distance between the 2D points and the mean circle centered at c=(xc, yc) """
    Ri = calculate_radius_C52(*c, x, y, centerX)
    return Ri - Ri.mean()

In [5]:
def get_circles_from_points(x, y, center, use_C52=True):
    # solve for top circle
    centerX0 = center[0]
    centerY = center[1]
    if use_C52:
        center_estimate, ier = optimize.leastsq(fit_circle_C52,centerY,args=(x,y,centerX0),col_deriv=True)
        Ri                   = calculate_radius_C52(*center_estimate,x,y,centerX)
        x_estimate = centerX0
        y_estimate = center_estimate[0]
        r_estimate = Ri.mean()
    else:
        center_estimate, ier = optimize.leastsq(fit_circle,center,args=(x,y),col_deriv=True)
        Ri                = calculate_radius(*center_top_estimate,x,y)
        x_estimate = center_estimate[0]
        y_estimate = center_estimate[1]
        r_estimate = Ri.mean()
    return x_estimate, y_estimate, r_estimate

In [6]:
# Compute a least squares estimate the top and bottom circles from lists of points along the top and bottom curves
#   for every ultrasound video (represented by a mean 2D image)

centerX = 0
for i in range(3):
    im = (im_mask[i]>0).astype(np.uint8)
    c = scipy.ndimage.measurements.center_of_mass(im)[1]
    centerX += c
    print(c)
centerX = int(np.round(centerX/3,0))

centert = (centerX, -55)
centerb = (centerX, -721)

x_top_estimate = np.zeros(num_mask)
y_top_estimate = np.zeros(num_mask)
r_top_estimate = np.zeros(num_mask)
x_bottom_estimate = np.zeros(num_mask)
y_bottom_estimate = np.zeros(num_mask)
r_bottom_estimate = np.zeros(num_mask)

for i in range(3):
    xt,yt,xb,yb = get_top_and_bottom_curve_points_C52(im_mask[i], centerX)
    
    x_top_estimate[i], y_top_estimate[i], r_top_estimate[i] = get_circles_from_points(xt, yt, centert, use_C52=True)
    x_bottom_estimate[i], y_bottom_estimate[i], r_bottom_estimate[i] = get_circles_from_points(xb, yb, centerb, use_C52=True)

    
    print(im_mask_depth[i])
    print("   Top: ", x_top_estimate[i], y_top_estimate[i], r_top_estimate[i])
    print("   Bottom: ", x_bottom_estimate[i], y_bottom_estimate[i], r_bottom_estimate[i])
    print()
    

952.6726509025195
955.0576735265394
953.6145025854096
5
   Top:  954.0 11.143492642914767 421.5909040997465
   Bottom:  954.0 -85.72432851471096 908.1117281864598

12
   Top:  954.0 27.99804506346475 227.3948431639985
   Bottom:  954.0 -336.26705460765976 1263.6184074778234

16
   Top:  954.0 -120.74286198368526 297.8172838772864
   Bottom:  954.0 162.20133192135734 889.5753071341097



In [7]:
def get_linear_map_C52(mask, xt, yt, rt, xb, yb, rb, ray_density = 0.5, blur = 0.5):
    try:
        assert(blur > 0)
    except:
        sys.exit("blur needs to be greater than zero")

    midline = xt 

    center_x = xt 
    center_y = yt

    center = np.array([center_x, center_y])

    inner_radius = rt
    outer_radius = (yb+rb)-yt

    radii = np.array([inner_radius, outer_radius])
    angle = (67.5/180)*math.pi
    
    left_angle = -angle/2
    right_angle = angle/2

    # determine the x and y sizes of the resampled image
    # from ray density. y size will be sector depth

    target_xsize = int(ray_density*(outer_radius)*angle + 0.5) # arc length (pixels) times ray density
    target_ysize = int(outer_radius - inner_radius + 0.5) # depth of US image

    # create mapping tensor

    mapping = np.zeros([target_ysize, target_xsize, 11])

    thetas = np.linspace(left_angle, left_angle+angle, target_xsize+2)
    rads = np.linspace(inner_radius, outer_radius, target_ysize+2)

    s0 = mask.shape[0]
    s1 = mask.shape[1]
    for i in range(target_xsize):
        print(i/target_xsize)
        for j in range(target_ysize): 
            
            theta = thetas[i + 1]
            rad = rads[j + 1]
            
            x = np.sin(theta)*rad + center_x
            y = np.cos(theta)*rad + center_y
            
            kernel_center_x = int(np.round(x))
            kernel_center_y = int(np.round(y))

            kernel_weights = np.zeros([3,3])

            G = multivariate_normal([x,y], np.eye(2)*blur)
            for m in range(3):
                for n in range(3):
                    i0 = kernel_center_x + m - 1
                    i1 = kernel_center_y + n - 1
                    if (i0<s0 and i1<s1 and mask[i0,i1]):
                        kernel_weights[m,n] = G.pdf([i0,i1])
                    else:
                        kernel_weights[m,n] = 0.0

            if (np.sum(kernel_weights) != 0):
                kernel_weights = kernel_weights / np.sum(kernel_weights)
            kernel_weights = kernel_weights.reshape(9)
            mapping[j,i] = np.concatenate(([kernel_center_x, kernel_center_y], kernel_weights))
        
    return mapping

In [11]:
for i in range(len(im_mask)):
    xt = x_top_estimate[i]
    yt = y_top_estimate[i]
    rt = r_top_estimate[i]
    xb = x_bottom_estimate[i]
    yb = y_bottom_estimate[i]
    rb = r_bottom_estimate[i]
    im_map = get_linear_map_C52(im_mask[i], xt, yt, rt, xb, yb, rb, ray_density = 0.5, blur = 0.5)
    im_map_img = itk.GetImageFromArray(im_map.astype('f'))
    filename = "linear_map_depth" + str(im_mask_depth[i]) + ".mha"
    itk.imwrite(im_map_img,filename)

0.0
0.0020920502092050207
0.0041841004184100415
0.006276150627615063
0.008368200836820083
0.010460251046025104
0.012552301255230125
0.014644351464435146
0.016736401673640166
0.01882845188284519
0.02092050209205021
0.02301255230125523
0.02510460251046025
0.027196652719665274
0.029288702928870293
0.03138075313807531
0.03347280334728033
0.03556485355648536
0.03765690376569038
0.0397489539748954
0.04184100418410042
0.043933054393305436
0.04602510460251046
0.04811715481171548
0.0502092050209205
0.05230125523012552
0.05439330543933055
0.056485355648535567
0.058577405857740586
0.060669456066945605
0.06276150627615062
0.06485355648535565
0.06694560669456066
0.06903765690376569
0.07112970711297072
0.07322175732217573
0.07531380753138076
0.07740585774058577
0.0794979079497908
0.08158995815899582
0.08368200836820083
0.08577405857740586
0.08786610878661087
0.0899581589958159
0.09205020920502092
0.09414225941422594
0.09623430962343096
0.09832635983263599
0.100418410041841
0.10251046025104603
0.1046

0.891213389121339
0.893305439330544
0.895397489539749
0.897489539748954
0.899581589958159
0.9016736401673641
0.9037656903765691
0.9058577405857741
0.9079497907949791
0.9100418410041841
0.9121338912133892
0.9142259414225942
0.9163179916317992
0.9184100418410042
0.9205020920502092
0.9225941422594143
0.9246861924686193
0.9267782426778243
0.9288702928870293
0.9309623430962343
0.9330543933054394
0.9351464435146444
0.9372384937238494
0.9393305439330544
0.9414225941422594
0.9435146443514645
0.9456066945606695
0.9476987447698745
0.9497907949790795
0.9518828451882845
0.9539748953974896
0.9560669456066946
0.9581589958158996
0.9602510460251046
0.9623430962343096
0.9644351464435147
0.9665271966527197
0.9686192468619247
0.9707112970711297
0.9728033472803347
0.9748953974895398
0.9769874476987448
0.9790794979079498
0.9811715481171548
0.9832635983263598
0.9853556485355649
0.9874476987447699
0.9895397489539749
0.9916317991631799
0.9937238493723849
0.99581589958159
0.997907949790795
0.0
0.00188679245283

0.7113207547169811
0.7132075471698113
0.7150943396226415
0.7169811320754716
0.7188679245283018
0.720754716981132
0.7226415094339622
0.7245283018867924
0.7264150943396226
0.7283018867924528
0.730188679245283
0.7320754716981132
0.7339622641509433
0.7358490566037735
0.7377358490566037
0.7396226415094339
0.7415094339622641
0.7433962264150943
0.7452830188679245
0.7471698113207547
0.7490566037735849
0.7509433962264151
0.7528301886792453
0.7547169811320755
0.7566037735849057
0.7584905660377359
0.7603773584905661
0.7622641509433963
0.7641509433962265
0.7660377358490567
0.7679245283018868
0.769811320754717
0.7716981132075472
0.7735849056603774
0.7754716981132076
0.7773584905660378
0.779245283018868
0.7811320754716982
0.7830188679245284
0.7849056603773585
0.7867924528301887
0.7886792452830189
0.7905660377358491
0.7924528301886793
0.7943396226415095
0.7962264150943397
0.7981132075471699
0.8
0.8018867924528302
0.8037735849056604
0.8056603773584906
0.8075471698113208
0.809433962264151
0.81132075471

0.3921852387843705
0.39363241678726485
0.3950795947901592
0.39652677279305354
0.3979739507959479
0.3994211287988423
0.40086830680173663
0.402315484804631
0.4037626628075253
0.40520984081041966
0.40665701881331406
0.4081041968162084
0.40955137481910275
0.4109985528219971
0.41244573082489144
0.41389290882778584
0.4153400868306802
0.41678726483357453
0.4182344428364689
0.4196816208393632
0.4211287988422576
0.42257597684515197
0.4240231548480463
0.42547033285094066
0.426917510853835
0.4283646888567294
0.42981186685962375
0.4312590448625181
0.43270622286541244
0.4341534008683068
0.4356005788712012
0.4370477568740955
0.4384949348769899
0.4399421128798842
0.44138929088277856
0.44283646888567296
0.4442836468885673
0.44573082489146165
0.447178002894356
0.44862518089725034
0.45007235890014474
0.4515195369030391
0.45296671490593343
0.4544138929088278
0.4558610709117221
0.4573082489146165
0.45875542691751087
0.4602026049204052
0.46164978292329956
0.4630969609261939
0.4645441389290883
0.46599131693